In [1]:
import altair as alt
import pandas as pd
import glob

In [2]:
def findHeaderIntPos(phyloFlashHeader: pd.DataFrame, frontOffset=12) -> int:
    # full que implementation might be faster with larger integer that has more digits
    recreateIntStack = []
    middleIndexSortValue = ""
    for char in phyloFlashHeader.loc[0,"PhyloFlash header"][frontOffset:]:
        if char.isdigit():
            recreateIntStack.append(char)
        elif len(recreateIntStack) > 0:
            break
    while(len(recreateIntStack) > 0):
        middleIndexSortValue += recreateIntStack.pop(0)
    return int(middleIndexSortValue)

In [3]:
# calculate percentages

files = glob.glob("./results/mbarc26/hd/*.csv")
df = []
for f in files:
    csv = pd.read_csv(f)
    csv["Percent"] = round(100*csv["HD for closest hit"] / csv[["Target sequence length", "Reference sequence length"]].min(axis=1), 1)
    csv.sort_values("Percent", inplace=True)
    df.append(csv)


In [4]:
# sort by phyloFlash header
sortedDF = sorted(df, key=findHeaderIntPos)
df = pd.concat(sortedDF, ignore_index=True, sort=False)

In [5]:
# trim full species names to match original heatmap names
dfTrimmedNames = df.copy()

dfTrimmedNames["Closest hit to reference 16S"] = dfTrimmedNames["Closest hit to reference 16S"].str.extract("([a-zA-Z\.]+\s[a-zA-Z]+)")

# manually adjust names that do not match substring extraction
dfTrimmedNames.loc[dfTrimmedNames["Closest hit to reference 16S"] == "E.coli K", "Closest hit to reference 16S"] = "E.coli"
dfTrimmedNames.loc[dfTrimmedNames["Closest hit to reference 16S"] == "Clostridium perfringensATCC", "Closest hit to reference 16S"] = "Clostridium perfringens"
dfTrimmedNames.loc[dfTrimmedNames["Closest hit to reference 16S"] == "Clostridium thermocellumVPI", "Closest hit to reference 16S"] = "Clostridium thermocellum"

In [6]:
# The original heatmaps sometimes have a specific arraignment

# explicitFullNameOrder = [
#     "Streptococcus pyogenes M1 GAS",#
#     "Hirschia baltica ATCC 49814",#
#     "Frateuria aurantia DSM 6220",#
#     "Thermobacillus composti KWC4",#
#     "Coraliomargarita akajimensis DSM 45221",#
#     "Pseudomonas stutzeri RCH2",#
#     "Natronobacterium gregoryi SP2",#
#     "Spirochaeta smaragdinae DSM 11293",#
#     "Clostridium perfringensATCC 13124",#
#     "Salmonella enterica subsp. arizonae serovar 62 z4 z23 - strain RSK2980",#
#     "Segniliparus rotundus DSM 44985",#
#     "Olsenella uli DSM 7084",#
#     "Natronococcus occultus DSM 3396",#
#     "Terriglobus roseus DSM 18391",#
#     "Halovivax ruber XH-70",#
#     "Corynebacterium glutamicum ATCC 13032",#
#     "Desulfosporosinus acidiphilus SJ4 DSM 22704",#
#     "Clostridium thermocellumVPI 7372 ATCC 27405",#
#     "Echinicola vietnamensis DSM 17526",#
#     "Meiothermus silvanus DSM 9946",#
#     "Desulfosporosinus meridiei DSM 13257",#
#     "Desulfotomaculum gibsoniae DSM 7213",#
#     "E.coli K12 ATCC 700926",#
#     "Salmonella bongori NCTC 12419",#
#     "Fervidobacterium pennivorans DSM 9078",#
#     "Nocardiopsis dassonvillei DSM 43111",
# ]
explicitNameOrder = [
    "Streptococcus pyogenes",
    "Hirschia baltica",
    "Frateuria aurantia",
    "Thermobacillus composti",
    "Coraliomargarita akajimensis",
    "Pseudomonas stutzeri",
    "Natronobacterium gregoryi",
    "Spirochaeta smaragdinae",
    "Clostridium perfringens",
    "Salmonella enterica",
    "Segniliparus rotundus",
    "Olsenella uli",
    "Natronococcus occultus",
    "Terriglobus roseus",
    "Halovivax ruber",
    "Corynebacterium glutamicum",
    "Desulfosporosinus acidiphilus",
    "Clostridium thermocellum",
    "Echinicola vietnamensis",
    "Meiothermus silvanus",
    "Desulfosporosinus meridiei",
    "Desulfotomaculum gibsoniae",
    "E.coli",
    "Salmonella bongori",
    "Fervidobacterium pennivorans",
    "Nocardiopsis dassonvillei",
]

In [10]:
base = alt.Chart(
    dfTrimmedNames.copy(),
    # title="Hamming Distances of assembled 16Ss to the true 16Ss for mock MBARC-26"
    ).encode(
    x=alt.X(
        "Closest hit to reference 16S:O",
        scale=alt.Scale(paddingInner=0),
        sort=explicitNameOrder,
        axis=alt.Axis()
    ),
    y=alt.Y(
        "PhyloFlash header:O",
        scale=alt.Scale(paddingInner=0),
        sort="x",
        title="Assembly"
    )
).properties(
    width= 1050,
    height=850
)

heatmap = base.mark_rect().encode(
    color=alt.Color(
        "Percent:Q",
        scale=alt.Scale(scheme="magma", reverse=True),
    )
)

text = base.mark_text(
    baseline="middle",
    fontSize=14
    ).encode(
        text="Percent:Q",
        color=alt.condition(
            alt.datum.Percent < 20,
            alt.value("lightgrey"),
            alt.value("black"),
        )
)

heat_plot = heatmap + text

heat_plot.configure_axis(
    titleFontSize = 23,
    labelFontSize = 13
).configure_title(
    fontSize=30
)


SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.core.Scale, validating 'additionalProperties'

        Additional properties are not allowed ('reverse' was unexpected)
        